In [16]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression



import warnings
warnings.filterwarnings('ignore')


In [17]:
df = np.round(pd.read_csv('50_Startups.csv')[['R&D Spend','Administration','Marketing Spend','Profit']]/10000)
np.random.seed(9)
df = df.sample(5)
df

,R&D Spend,Administration,Marketing Spend,Profit
21,8.0,15.0,30.0,11.0
37,4.0,5.0,20.0,9.0
2,15.0,10.0,41.0,19.0
14,12.0,16.0,26.0,13.0
44,2.0,15.0,3.0,7.0


### What is the "Sequence" of a Seed?
When we say a seed generates a "sequence," we mean that random number generators in computers are actually **deterministic algorithms**. They don't create true randomness; they just produce a long list of numbers based on a math formula.

Start the random-number generator from position 9 in its internal sequence.

A seed is just an integer that selects where the pseudo-random sequence begins.

In [18]:
df = df.iloc[:, :-1]
df

,R&D Spend,Administration,Marketing Spend
21,8.0,15.0,30.0
37,4.0,5.0,20.0
2,15.0,10.0,41.0
14,12.0,16.0,26.0
44,2.0,15.0,3.0


In [34]:
# assigning missing NaN values at different cells
df.iloc[1,0] = np.nan
df.iloc[3,1] = np.nan
df.iloc[-1,-1] = np.nan

df.sample(5)

,R&D Spend,Administration,Marketing Spend
2,15.0,10.0,41.0
21,8.0,15.0,30.0
37,NaN,5.0,20.0
44,2.0,15.0,NaN
14,12.0,NaN,26.0


### Type 1 - using mean

In [44]:
df1 = pd.DataFrame()

df1['R&D Spend'] = df['R&D Spend'].fillna(df['R&D Spend'].mean())
df1['Administration'] = df['Administration'].fillna(df['Administration'].mean())
df1['Marketing Spend'] = df['Marketing Spend'].fillna(df['Marketing Spend'].mean())

df1


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,9.25,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.25,26.00
44,2.00,15.00,29.25
